In [6]:
import os
import pandas as pd
import numpy as np
import sweetviz as sv

In [23]:
#!pip install sweetviz

In [21]:
#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_LDI_II
BASE_DIR = '../'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "PetFinder/input/train/train.csv")

#Reporte EDA
PATH_TO_EDA_REPORT = os.path.join(BASE_DIR, "EDA/EDA_report.html")

## Importar conjunto de datos

In [13]:
dataset = pd.read_csv(PATH_TO_TRAIN)

## Aplicar data engineering

In [17]:
def feature_engineering(dataset):
    # Categorizacion de edad
    bins = [0, 12, 60, float('inf')]  # Define la cantidad de categorias y los limites de las mismas
    labels = [1, 2, 3]  # 1='puppy' 2='adult' 3='senior'
    dataset['AgeCategory'] = pd.cut(dataset['Age'], bins, labels=labels, right=False)

    # Combinacion para Breed
    dataset['Breed'] = dataset.apply(lambda row: 2 if row['Breed2'] != 0 else 1, axis=1)

    # Color
    dataset['IsMultiColored'] = dataset.apply(lambda row: 1 if row['Color2'] != 0 or row['Color3'] != 0 else 0, axis=1)

    # Name
    dataset['Name'] = dataset['Name'].fillna('No Name Yet')
    dataset['Name'] = dataset['Name'].replace('', 'No Name Yet')
    dataset['name_code'] = dataset['Name'].apply(lambda x: 1 if x != 'No Name Yet' else 0)

    # Creación de nuevas características
    dataset['Age_log'] = np.log1p(dataset['Age'])  # logaritmo de la edad
    dataset['PhotoAmt_log'] = np.log1p(dataset['PhotoAmt'])  # logaritmo de la cantidad de fotos
    dataset['VideoAmt_log'] = np.log1p(dataset['VideoAmt'])  # logaritmo de la cantidad de videos

    # Interacciones entre características
    dataset['Age_PhotoAmt'] = dataset['Age'] * dataset['PhotoAmt']  # relación entre la edad de la mascota y la cantidad de fotos
    dataset['Age_VideoAmt'] = dataset['Age'] * dataset['VideoAmt']  # relación entre la edad de la mascota y la cantidad de videos

    # Agregaciones y agrupaciones
    dataset['Breed1_mean_Age'] = dataset.groupby('Breed1')['Age'].transform('mean')  # promedio de edad por raza
    dataset['Breed1_mean_PhotoAmt'] = dataset.groupby('Breed1')['PhotoAmt'].transform('mean')  # promedio de fotos por raza

    # Agregar cantidad de palabras del describe y longitud del texto
    dataset["description_word_count"] = dataset["Description"].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
    dataset['Description_Length'] = dataset['Description'].apply(lambda x: len(x) if isinstance(x, str) else 0)

    # Health
    dataset['Health'] = dataset['Vaccinated'].astype(str) + '_' + dataset['Dewormed'].astype(str) + '_' + dataset['Sterilized'].astype(str)

    # Nuevas características basadas en interacciones y transformaciones
    dataset['Photo_Video_Ratio'] = dataset['PhotoAmt'] / (dataset['VideoAmt'] + 1)  # Evitar división por cero
    dataset['Fee_log'] = np.log1p(dataset['Fee'])  # Logaritmo de la tarifa de adopción
    dataset['Fee_Age_Ratio'] = dataset['Fee'] / (dataset['Age'] + 1)  # Evitar división por cero
    dataset['RescuerID_count'] = dataset.groupby('RescuerID')['RescuerID'].transform('count')  # Número de mascotas rescatadas por el mismo rescatador
    dataset['Name_Length'] = dataset['Name'].apply(len)  # Longitud del nombre de la mascota
    dataset['Name_Word_Count'] = dataset['Name'].apply(lambda x: len(x.split()))  # Número de palabras en el nombre de la mascota
    dataset['RescuerID_Length'] = dataset['RescuerID'].apply(len)  # Longitud del ID del rescatador
    dataset['RescuerID_Word_Count'] = dataset['RescuerID'].apply(lambda x: len(x.split()))  # Número de palabras en el ID del rescatador
    dataset['MaturitySize_FurLength'] = dataset['MaturitySize'] * dataset['FurLength']
    dataset['PhotoAmt_FurLength'] = dataset['PhotoAmt'] * dataset['FurLength']
    dataset['VideoAmt_FurLength'] = dataset['VideoAmt'] * dataset['FurLength']

    return dataset

In [18]:
# aplicamos feature engineering
dataset = feature_engineering(dataset)
dataset.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Fee_log,Fee_Age_Ratio,RescuerID_count,Name_Length,Name_Word_Count,RescuerID_Length,RescuerID_Word_Count,MaturitySize_FurLength,PhotoAmt_FurLength,VideoAmt_FurLength
0,2,No Name Yet,3,299,0,1,1,7,0,1,...,4.615121,25.0,8,11,3,32,1,1,1.0,0
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,0.000000,0.0,1,11,3,32,1,4,4.0,0
2,1,No Name Yet,1,307,0,1,2,7,0,2,...,0.000000,0.0,459,11,3,32,1,4,14.0,0
3,1,No Name Yet,4,307,0,2,1,2,0,2,...,5.017280,30.0,50,11,3,32,1,2,8.0,0
4,1,No Name Yet,1,307,0,1,1,0,0,2,...,0.000000,0.0,134,11,3,32,1,2,3.0,0


## Generar AUTO-EDA

In [20]:
report = sv.analyze(dataset)

                                             |          | [  0%]   00:00 -> (? left)

In [22]:
report.show_html(filepath=PATH_TO_EDA_REPORT, open_browser=False)

Report ../EDA/EDA_report.html was generated.
